# File to obtain Galaxy data needed
need:
- coordinates
- group number
- stellar mass
- luminosities
- radii?

- over timescale

## Some important constants

In [1]:
%load_ext autoreload 
#hopefully this will reload the modules when they are changed specifically when i change the plotting modules
%autoreload 2

In [2]:
sim = 'TNG300-1'
TNG300_1_boxsize = 	302.6 #Mpc
scratchDataDirc = f'/scratch/poulin.al/lopsided/{sim}/data'

In [3]:
from myproject.utilities import iapi_TNG, Subhalo, GalaxyGroup, ListGalaxyGroup
import os
import h5py as h5
import numpy as np

In [4]:
baseUrl = 'http://www.tng-project.org/api/'
r=iapi_TNG.get(baseUrl)
print(r)
#check the properties of the simulation you have selected
simUrl = baseUrl+sim
print(simUrl) 
simdata : dict[str, object] = iapi_TNG.get(simUrl)
print(simdata['description'])
simdata.keys()

Fetching data from http://www.tng-project.org/api/ with params None and saving to temp
Response code: 200
{'simulations': [{'name': 'Illustris-1', 'num_snapshots': 134, 'url': 'http://www.tng-project.org/api/Illustris-1/'}, {'name': 'Illustris-1-Dark', 'num_snapshots': 136, 'url': 'http://www.tng-project.org/api/Illustris-1-Dark/'}, {'name': 'Illustris-2', 'num_snapshots': 136, 'url': 'http://www.tng-project.org/api/Illustris-2/'}, {'name': 'Illustris-2-Dark', 'num_snapshots': 136, 'url': 'http://www.tng-project.org/api/Illustris-2-Dark/'}, {'name': 'Illustris-3', 'num_snapshots': 136, 'url': 'http://www.tng-project.org/api/Illustris-3/'}, {'name': 'Illustris-3-Dark', 'num_snapshots': 136, 'url': 'http://www.tng-project.org/api/Illustris-3-Dark/'}, {'name': 'TNG100-1', 'num_snapshots': 100, 'url': 'http://www.tng-project.org/api/TNG100-1/'}, {'name': 'TNG100-1-Dark', 'num_snapshots': 100, 'url': 'http://www.tng-project.org/api/TNG100-1-Dark/'}, {'name': 'TNG100-2', 'num_snapshots': 100

dict_keys(['name', 'description', 'name_alt', 'boxsize', 'z_start', 'z_final', 'cosmology', 'omega_0', 'omega_L', 'omega_B', 'hubble', 'physics_model', 'has_cooling', 'has_starformation', 'has_winds', 'has_blackholes', 'mass_gas', 'mass_dm', 'softening_dm_comoving', 'softening_stars_comoving', 'softening_blackholes_comoving', 'softening_gas_comoving', 'softening_dm_max_phys', 'softening_stars_max_phys', 'softening_blackholes_max_phys', 'softening_gas_max_phys', 'softening_gas_factor', 'softening_gas_comoving_min', 'num_dm', 'num_tr_mc', 'num_tr_vel', 'longids', 'is_uniform', 'is_zoom', 'is_subbox', 'num_files_snapshot', 'num_files_groupcat', 'num_files_rockstar', 'num_files_lhalotree', 'num_files_sublink', 'num_files_ctrees', 'filesize_lhalotree', 'filesize_sublink', 'filesize_ctrees', 'filesize_ics', 'filesize_simulation', 'has_fof', 'has_subfind', 'has_rockstar', 'has_lhalotree', 'has_sublink', 'has_ctrees', 'permission_required', 'num_snapshots', 'url', 'parent_simulation', 'child_s

In [5]:
h = simdata.get('hubble')
print(f"hubble: {h}")

z = 0
a = 1.0 / (1 + z)  # scale factor, where z = redshift

hubble: 0.6774


## Now get

### subhalo data

In [8]:
if not os.path.exists(scratchDataDirc + 'catalogs'):
    os.makedirs(scratchDataDirc + 'catalogs')
    print(f'created directory: {scratchDataDirc} "catalogs"')
if not os.path.exists(scratchDataDirc + 'catalogs/SubhaloFlag'):
    os.makedirs(scratchDataDirc + 'catalogs/SubhaloFlag')
    print(f'created directory: {scratchDataDirc} "catalogs/SubhaloFlag"')
flag=iapi_TNG.getSubhaloField('SubhaloFlag',simulation=sim,fileName=scratchDataDirc+'catalogs/SubhaloFlag/SubhaloFlag',rewriteFile=0)

In [9]:
print(scratchDataDirc+'catalogs/SubhaloFlag/SubhaloFlag')

/scratch/poulin.al/lopsided/TNG300-1/datacatalogs/SubhaloFlag/SubhaloFlag


In [10]:
#let's fetch a field that will tell us about the mass of the galaxy
#SubhaloMassType gives the total mass of all bound particles, separated by particle type
if not os.path.exists(scratchDataDirc + 'catalogs/MassType'):
    os.makedirs(scratchDataDirc + 'catalogs/MassType')
    print(f'created directory: {scratchDataDirc} "catalogs/MassType"')
mass=iapi_TNG.getSubhaloField('SubhaloMassType',simulation=sim,fileName=scratchDataDirc+'catalogs/MassType/MassType',rewriteFile=0) # in 10^10 solar masses/h
print(mass.shape)

#Pull the stellar mass: 
print(mass[:,3])
stellar_mass=mass[:,4]
print(stellar_mass)

stellar_mass=stellar_mass*10**10/h #convert to one solar masses

(14485709, 6)
[0. 0. 0. ... 0. 0. 0.]
[530.9484   253.66284   57.365696 ...   0.         0.         0.      ]


In [11]:
if not os.path.exists(scratchDataDirc + 'catalogs/SubhaloGrNr'):
    os.makedirs(scratchDataDirc + 'catalogs/SubhaloGrNr')
    print(f'created directory: {scratchDataDirc} "catalogs/SubhaloGrNr"')
subhaloGroupNum = iapi_TNG.getSubhaloField('SubhaloGrNr',simulation = sim,fileName=scratchDataDirc+'catalogs/SubhaloGrNr/SubhaloGrNr',rewriteFile=0)

In [12]:
if not os.path.exists(scratchDataDirc + 'catalogs/SubhaloPos'):
    os.makedirs(scratchDataDirc + 'catalogs/SubhaloPos')
    print(f'created directory: {scratchDataDirc} "catalogs/SubhaloPos"')
subhaloPos = iapi_TNG.getSubhaloField('SubhaloPos',simulation = sim,fileName=scratchDataDirc+'catalogs/SubhaloPos/SubhaloPos',rewriteFile=0) # in ckpc/h

## convert positions from comoving to physical kpc
subhaloPos = subhaloPos * a / h  # kpc

In [13]:
if not os.path.exists(scratchDataDirc + 'catalogs/SubhaloHalfmassRad'):
    os.makedirs(scratchDataDirc + 'catalogs/SubhaloHalfmassRad')
    print(f'created directory: {scratchDataDirc} "catalogs/SubhaloHalfmassRad"')
subhaloHalfmassRad = iapi_TNG.getSubhaloField('SubhaloHalfmassRad',simulation = sim,fileName=scratchDataDirc+'catalogs/SubhaloHalfmassRad/SubhaloHalfmassRad',rewriteFile=0) # in ckpc/h

## convert to kpc
subhaloHalfmassRad = subhaloHalfmassRad * a / h  # kpc

In [14]:
if not os.path.exists(scratchDataDirc + 'catalogs/SubhaloVmaxRad'):
    os.makedirs(scratchDataDirc + 'catalogs/SubhaloVmaxRad')
    print(f'created directory: {scratchDataDirc} "catalogs/SubhaloVmaxRad"')
SubhaloVmaxRad = iapi_TNG.getSubhaloField('SubhaloVmaxRad',simulation = sim,fileName=scratchDataDirc+'catalogs/SubhaloVmaxRad/SubhaloVmaxRad',rewriteFile=0) # in ckpc/h

## convert to kpc
SubhaloVmaxRad = SubhaloVmaxRad * a / h  # kpc

In [15]:
if not os.path.exists(scratchDataDirc + 'catalogs/SubhaloStellarPhotometrics'):
    os.makedirs(scratchDataDirc + 'catalogs/SubhaloStellarPhotometrics')
    print(f'created directory: {scratchDataDirc} "catalogs/SubhaloStellarPhotometrics"')
SubhaloStellarPhotometrics = iapi_TNG.getSubhaloField('SubhaloStellarPhotometrics',simulation = sim,fileName=scratchDataDirc+'catalogs/SubhaloStellarPhotometrics/SubhaloStellarPhotometrics',rewriteFile=0) # Eight bands: U, B, V, K, g, r, i, z, all in mag

### Group data

In [16]:
if not os.path.exists(scratchDataDirc + 'catalogs/GroupCM'):
    os.makedirs(scratchDataDirc + 'catalogs/GroupCM')
    print(f'created directory: {scratchDataDirc} "catalogs/GroupCM"')
groupCM = iapi_TNG.getHaloField('GroupCM',simulation = sim,fileName=scratchDataDirc+'catalogs/GroupCM/GroupCM',rewriteFile=0) # in ckpc/h

## convert to kpc
groupCM = groupCM * a / h  # kpc


In [17]:
if not os.path.exists(scratchDataDirc + 'catalogs/GroupMCrit200'):
    os.makedirs(scratchDataDirc + 'catalogs/GroupMCrit200')
    print(f'created directory: {scratchDataDirc} "catalogs/GroupMCrit200"')
groupMCrit200 = iapi_TNG.getHaloField('Group_M_Crit200',simulation = sim,fileName=scratchDataDirc+'catalogs/GroupMCrit200/GroupMCrit200',rewriteFile=0) # in 10^10 solar masses/h

## convert to solar masses
groupMCrit200 = groupMCrit200 * 1e10 / h  # solar masses

In [18]:
if not os.path.exists(scratchDataDirc + 'catalogs/GroupPos'):
    os.makedirs(scratchDataDirc + 'catalogs/GroupPos')
    print(f'created directory: {scratchDataDirc} "catalogs/GroupPos"')
groupPos = iapi_TNG.getHaloField('GroupPos',simulation = sim,fileName=scratchDataDirc+'catalogs/GroupPos/GroupPos',rewriteFile=0) # in ckpc/h

## convert to kpc
groupPos = groupPos * a / h  # kpc

## transform into custom classes

In [19]:
headerInformation = {
    'simulation': sim,
    'description': simdata.get('description'),
    'boxsize_Mpc': TNG300_1_boxsize,
    'hubble_param': h,
    'redshift': z
}

In [41]:
list_of_galaxy_groups : ListGalaxyGroup = ListGalaxyGroup(headerInformation = headerInformation, listGalaxyGroups=[])

temp_dict_galaxy_groups : dict[int, GalaxyGroup] = {} #temporary dictionary to hold galaxy groups while we build them since lookup is faster

print(f"num subhalos: {subhaloGroupNum.shape[0]}")
print(f"rough number of galaxygroups: {np.unique(subhaloGroupNum)}")
for i in range(subhaloGroupNum.shape[0]):
    group_num = subhaloGroupNum[i]
    # print(f"GroupNum: {group_num}")
    # if i > 5:
    #     break
    if group_num not in temp_dict_galaxy_groups: #check if galaxy group already exists - if not, create it - else just add the subhalo to it
        #initialize new empty galaxy group
        galaxyGroup = None
        galaxyGroup = GalaxyGroup(group_id=group_num, posCM=groupCM[group_num], MCrit200=groupMCrit200[group_num], pos=groupPos[group_num], listSubhalos=[])
        # print(galaxyGroup.getNumSubhalos())
        temp_dict_galaxy_groups[group_num] = galaxyGroup
        
        list_of_galaxy_groups.addGalaxyGroup(galaxyGroup) #add to the master list
    # print(galaxyGroup.getNumSubhalos())
        
    subhalo = Subhalo(i, group_id=group_num, flag=flag[i], mass=mass[i], stellarMass=stellar_mass[i], groupNumber=subhaloGroupNum[i], position=subhaloPos[i], halfMassRad=subhaloHalfmassRad[i], vmaxRadius=SubhaloVmaxRad[i], luminosities=SubhaloStellarPhotometrics[i]) #create subhalo
    
    temp_dict_galaxy_groups[group_num].addSubhalo(subhalo) #add subhalo to the appropriate galaxy group

print(f'Constructed ListGalaxyGroup with {list_of_galaxy_groups.getNumGalaxyGroups()} galaxy groups.')
    

num subhalos: 14485709
rough number of galaxygroups: [       0        1        2 ... 17625886 17625887 17625888]
Constructed ListGalaxyGroup with 11504190 galaxy groups.


## Tweak data
Filter the galaxies
- remove non cosmlogoical in origin (subhaloflag = 0)
- only mass greater than 13 solar mass
- remove groups where no single central (i.e. central pos does not match cm pos)
- Later on will need to filter based on radius due to smoothing length: because the smoothing length used by the TNG300 is different for z < 1 and z >= 1 we'll likely want to adopt a minimum  size for the galaxies we want to use following Curtis et al. (2026) (note: we need to check to see if TNGCluster used the same smoothing length change as TNG300)

Correct data
- use the subhalo magnitudes from Nelson et al. (2018)
to assign luminosities to the galaxies because these include the effects
of dust obscuration and better approximate what we'd see in the real
universe (e.g., the Sloan Digital Sky Survey)
- correct the stellar masses of the galaxies following
Appendix A in Pillepich et al. (2018) because the stellar masses 
aren't as well converged in TNG300 as in the benchmark TNG100 simulation
- correct positions to be relative to central and correct for periodic boundary conditions

In [49]:
print(list_of_galaxy_groups.getGalaxyGroupI(0).getNumSubhalos())
print(list_of_galaxy_groups.getGalaxyGroupI(0).getSubhaloI(5).getFlag())
print(list_of_galaxy_groups.getGalaxyGroupI(1000).getNumSubhalos())

11748
True
330


In [ ]:
# list_of_galaxy_groups.filterSubhalos_parallel(minStellarMass=1e9, maxStellarMass=None, minHalfMassRad_kpc=0, maxHalfMassRad_kpc=None, centralPosTolerance_kpc=1000)
# print(f'After filtering, ListGalaxyGroup has {list_of_galaxy_groups.getNumGalaxyGroups()} galaxy groups.')

# list_of_galaxy_groups.correctPositions_parallel(TNG300_1_boxsize)
# print('Corrected positions for periodic boundary conditions.')
list_of_galaxy_groups.filterAndCorrectSubhalos_parallel(minGGMass=1e14, maxGGMass=None, minSatStellarMass=None, maxSatStellarMass=None, minHalfMassRad_kpc=0, maxHalfMassRad_kpc=None, centralPosTolerance_kpc=1000, boxsize=TNG300_1_boxsize)
print(f'After filtering and correcting, ListGalaxyGroup has {list_of_galaxy_groups.getNumGalaxyGroups()} galaxy groups.')


## save into hdf5 file
Save the data we've collected (stellar mass, group number, flag, etc) into an HDF5 file for easy access and analysis later.

In [ ]:
output_filename = scratchDataDirc + f'/galaxy_data_{sim}.hdf5'
with h5.File(output_filename, 'w') as f:
    # list_of_galaxy_groups.save_to_hdf5(f)
    list_of_galaxy_groups.save_to_hdf5_parallel(f)
print(f'Saved galaxy data to {output_filename}')

## Parallel Processing Example

The following methods now have parallel versions that use multiprocessing for much faster execution:

### Available Parallel Methods:
- `filterSubhalos_parallel()` - Filter subhalos across galaxy groups
- `correctPositions_parallel()` - Correct positions for periodic boundaries
- `compute_all_pairwise_polar_differences_parallel()` - Compute pairwise differences
- `compute_all_MRL_directionality_parallel()` - Compute MRL values
- `save_to_hdf5_parallel()` - Serialize and save data (data prep is parallel, writing is sequential due to HDF5 limitations)

### Usage:
```python
# Serial (original):
list_of_galaxy_groups.filterSubhalos(minStellarMass=1e9, ...)
list_of_galaxy_groups.correctPositions(boxsize)
list_of_galaxy_groups.compute_all_pairwise_polar_differences()
list_of_galaxy_groups.compute_all_MRL_directionality()
with h5.File(filename, 'w') as f:
    list_of_galaxy_groups.save_to_hdf5(f)

# Parallel (faster):
list_of_galaxy_groups.filterSubhalos_parallel(minStellarMass=1e9, ...)
list_of_galaxy_groups.correctPositions_parallel(boxsize)
list_of_galaxy_groups.compute_all_pairwise_polar_differences_parallel()
list_of_galaxy_groups.compute_all_MRL_directionality_parallel()
with h5.File(filename, 'w') as f:
    list_of_galaxy_groups.save_to_hdf5_parallel(f)

# Or specify number of processes:
list_of_galaxy_groups.filterSubhalos_parallel(minStellarMass=1e9, n_processes=8, ...)
```

The parallel versions automatically use (CPU count - 1) processes by default.

**Note on save_to_hdf5_parallel:** HDF5 doesn't support concurrent writes, so the actual file writing is still sequential. However, data serialization (converting objects to dictionaries) is done in parallel, which can provide speedup for large datasets with complex structures.